In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

# Data loading

The universe of data consists of the 48 industry portfolios from the Kenneth R. French database, which consists of returns from 1926 to the first month of 2023. For purposes of this analysis, I used monthly returns.

In addition, was used the fama french factors database to extract the historical monthly risk free rate and the excess market return, which will be used to find the alpha and beta of the strategies with respect to CAPM. 

In [315]:
rets_ind = pd.read_csv(r"48_industry_portfolios_final.csv", index_col="Date", parse_dates=True)
rets_ind.columns = rets_ind.columns.str.strip()
rets_ind.replace(-99.99, np.nan, inplace=True) 
rets_ind = rets_ind/100

ff_factors = pd.read_csv(r"FF_research_data_factors.csv", index_col="Date", parse_dates=True)/100

excess_rets_ind = rets_ind.subtract(ff_factors["RF"], axis=0) #converting the industry portfolio returns to excess returns
excess_rets_ind.index = pd.to_datetime(excess_rets_ind.index, format="%Y%m")

ff_factors.index = pd.to_datetime(ff_factors.index, format="%Y%m")

excess_rets_ind

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,0.0242,0.0051,NaN,-0.0558,0.0145,0.0843,0.0103,0.4999,0.0223,-0.0090,...,0.0107,-0.0101,-0.2045,0.0156,0.0147,0.0474,-0.0116,0.0692,-0.0647,0.0239
1926-08-01,0.0012,0.0370,NaN,0.1900,0.0476,0.1656,-0.0010,0.4273,-0.0311,0.0454,...,0.0375,0.0478,0.0917,-0.0070,-0.0115,0.3398,0.0080,0.0277,-0.0042,0.0323
1926-09-01,-0.0338,0.0149,NaN,0.0540,0.0287,0.0810,0.0320,-0.0514,0.0153,-0.0639,...,-0.0687,-0.0023,0.0009,-0.0110,-0.0037,-0.0148,-0.0030,-0.0197,-0.0746,-0.0041
1926-10-01,0.0023,-0.0541,NaN,-0.0640,-0.0195,-0.0172,-0.0681,0.0505,-0.0330,-0.0144,...,-0.0522,-0.0266,-0.0653,-0.0365,-0.0237,-0.1145,-0.0378,-0.0404,-0.0409,-0.0671
1926-11-01,-0.0888,0.0758,NaN,0.1098,0.0927,-0.0031,0.0002,-0.0671,0.0134,0.0054,...,0.0229,0.0011,-0.0057,0.0103,0.0161,-0.0401,0.0148,-0.0039,0.0152,-0.0072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,-0.1443,-0.1342,-0.1914,-0.1346,-0.0981,-0.1590,-0.1257,-0.1972,-0.1439,-0.1656,...,-0.1257,-0.1216,-0.1192,-0.1439,-0.0804,-0.0593,-0.1007,-0.1432,-0.1051,-0.0703
2022-10-01,0.1335,0.0970,0.0258,0.0020,0.1184,0.0768,0.1068,0.0750,0.0555,0.0754,...,0.1048,0.0980,0.1115,0.0964,0.1406,0.0837,0.0862,0.0545,0.1077,0.0092
2022-11-01,-0.0340,-0.0208,-0.0348,-0.0679,0.0059,-0.0213,0.0450,0.1377,0.0584,0.0846,...,0.0826,0.0007,0.1019,0.0229,0.0026,0.0216,0.0109,0.0697,0.0601,-0.0389


# Functions used

In [377]:
def tickers_per_port(vol_per_ind, n_ind_portfolios):
    '''
    Given the industries sorted based on volatility and the number of industries/tickers, this function gives as output
    each portfolio with the name of the corresponding industries that compose each of them for a specific formation period.  
    '''
    industries_portfolios = {}
    n_portfolio = 1
    start = 0
    
    for n in n_ind_portfolios:
        industries = vol_per_ind.index[start:start+n]
        industries_portfolios[n_portfolio] = industries
        n_portfolio += 1
        start += n 
        
    return industries_portfolios

def rets_per_port(results, trading_data, ind_per_port):
    '''
    Given the returns from the trading period and the industries that compose each portfolio, this function provides
    as output the returns per portfolio in a specific trading period.
    '''
    for port in ind_per_port.keys():
        industries = ind_per_port[port]
        rets_port = trading_data[industries].mean(axis=1) #because it is equal weighted
        results.loc[trading_data.index, port] = rets_port
        
    return results

def drawdown(return_series: pd.Series):
    """
    Takes a time series of asset returns. Returns a DataFrame with columns for the wealth index, the previous peaks, and 
    the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

def beta_alpha(return_series: pd.Series, excess_mkt_ret, periods_per_year=12):
    """
    Takes a time series of asset returns, and runs a regression in which the excess market returns are the independent variable.
    The output of this function is the alpha and its corresponding t-value, and the beta and its corresponding t-value.
    """
    df = pd.merge(return_series, excess_mkt_ret, left_index=True, right_index=True)
    statement = f"{df.columns[0]}~{df.columns[1]}"
    results_regression = ols(statement, data = df).fit()
    alpha = results_regression.params[0].round(4)*periods_per_year
    t_alpha = results_regression.tvalues[0].round(4)
    beta = results_regression.params[1].round(4)
    t_beta = results_regression.tvalues[1].round(4)
    
    return alpha, t_alpha, beta, t_beta

def annualize_rets(r, periods_per_year):
    """
    Annualizes a set of returns
    """
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

def annualize_vol(r, periods_per_year):
    """
    Annualizes the vol of a set of returns
    """
    return r.std()*(periods_per_year**0.5)

def sharpe_ratio(r, riskfree_rate, periods_per_year):
    """
    Computes the annualized sharpe ratio of a set of returns
    """
    # convert the annual riskfree rate to per period
    rf_per_period = (1+riskfree_rate)**(1/periods_per_year)-1
    excess_ret = r - rf_per_period
    ann_ex_ret = annualize_rets(excess_ret, periods_per_year)
    ann_vol = annualize_vol(r, periods_per_year)
    return ann_ex_ret/ann_vol

def stats(r, periods_per_year, excess_mkt_ret, riskfree_rate=0):
    """
    Return a DataFrame that contains aggregated summary stats for the returns in the columns of r
    """
    ann_r = r.aggregate(annualize_rets, periods_per_year=periods_per_year)
    ann_vol = r.aggregate(annualize_vol, periods_per_year=periods_per_year)
    ann_sr = r.aggregate(sharpe_ratio, riskfree_rate=riskfree_rate, periods_per_year=periods_per_year)
    df_results = pd.DataFrame({
        "Annualized Return": ann_r,
        "Annualized Vol": ann_vol,
        "Sharpe Ratio": ann_sr,
    })
    df_beta_alpha = r.aggregate(beta_alpha, excess_mkt_ret=excess_mkt_ret, periods_per_year=periods_per_year)
    df_beta_alpha.index = ['alpha', 't_value', 'beta', 't_value']
    return pd.concat([df_results.T, df_beta_alpha])

In [317]:
def vol_strat(data_excess_rets = excess_rets_ind, formation_period = 36, n_portfolios = 10):
    '''
    This function backtests a strategy based on historical volatility and creates n number of portfolios. 
    trading period is set by default to 1 month.
    The output of this function is a dataframe of the monthly returns for each of the portfolios created.
    '''
    trading_period = 1
    start_period = 0
    n_periods = data_excess_rets.shape[0]
    results = pd.DataFrame(columns=np.arange(1,n_portfolios+1,1), index=excess_rets_ind.iloc[formation_period:].index) #Empty dataframe where the returns will be stored.
    
    
    while start_period+formation_period < n_periods:
        #If there is an NA value during the formation period, then the industry will be dropped as shown below.
        formation_data = data_excess_rets.iloc[start_period:start_period+formation_period,:].dropna(axis=1) #this is the formation period 
        trading_data = data_excess_rets[formation_data.columns].iloc[start_period+formation_period:start_period+formation_period+trading_period,:] #trading period based on the above formation period
        
        n_ind = len(formation_data.columns) #number of industries available to be traded in the trading period
        n_ind_portfolios = [n_ind // n_portfolios + (1 if x < n_ind % n_portfolios else 0) for x in range(n_portfolios)] #Based on the number of industries available, this is giving
        #a list that has the number of industries that will correspond to each portfolio
        
        vol_per_ind = formation_data.std().sort_values() #calculating the volatility of each industry during the dormation period and sorting in an ascending order
        ind_per_port = tickers_per_port(vol_per_ind=vol_per_ind, n_ind_portfolios=n_ind_portfolios) #this is a dictionary that includes as keys the decile of the portfolio
        #and as values a list of the names of the industries included in each portfolio
        
        trading_data.replace(np.nan, -1, inplace=True) #when there is an na in the trading data, then it is assumed that the NA means a 100% loss in that industry
        
        results = rets_per_port(results=results, trading_data=trading_data, ind_per_port=ind_per_port) #adds the return per portfolio for each date to the results dataframe
        
        start_period += trading_period
        
    return results

The vol_strat() function above is the backtestor which will allow me to compute the returns on all portfolios. At the end of every month, equally weighted decile portfolios are constructed based on the ranking of the stocks depending on their past 36 months volatility (3 years) of monthly returns. Industries with the lowest volatility are assigned to the top decile portfolio.

In [318]:
rets_vol_strat = vol_strat() #running the backtestor for the 10 portfolios and storing the results
rets_universe = vol_strat(n_portfolios=1) #running the backtestor for the whole universe
LongShort = rets_vol_strat[1]-rets_vol_strat[10] #finding the returns of the long short portfolio (top decile - bottom decile)
rets_vol_strat["D1-10"] = LongShort
rets_vol_strat["Univ"] = rets_universe
rets_vol_strat = rets_vol_strat.astype(float)
rets_vol_strat.columns = ["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D1_minus_ten", "Univ"] #changing the name of the columns

Below can be seen a dataframe with the monthly excess returns across all portfolios. Portfolios are rebalanced monthly and no transaction costs are considered for matters of this exercise.

In [302]:
rets_vol_strat

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D1-10,Univ
Date,,,,,,,,,,,,
1929-07-01,0.057400,0.025050,-0.014175,0.024975,0.056075,0.054925,0.001125,0.001450,0.004700,-0.027250,0.084650,0.018428
1929-08-01,0.049100,0.006225,0.018450,0.013275,0.057450,0.033025,0.039550,-0.016275,0.024325,0.029175,0.019925,0.025430
1929-09-01,-0.064025,-0.019825,-0.042300,-0.003375,-0.071125,-0.028300,-0.049750,-0.014950,-0.033325,-0.069625,0.005600,-0.039660
1929-10-01,-0.155150,-0.160450,-0.194950,-0.194000,-0.216475,-0.188250,-0.208825,-0.198900,-0.209950,-0.171150,0.016000,-0.189810
1929-11-01,-0.106100,-0.093725,-0.071100,-0.097950,-0.154250,-0.152350,-0.131350,-0.158850,-0.149450,-0.217775,0.111675,-0.133290
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,-0.119420,-0.115600,-0.122260,-0.131400,-0.130360,-0.129900,-0.162720,-0.113280,-0.139450,-0.100150,-0.019270,-0.126731
2022-10-01,0.101140,0.094000,0.072680,0.073720,0.056340,0.079280,0.105120,0.100860,0.083550,0.138500,-0.037360,0.089665
2022-11-01,0.002160,0.011500,0.036860,0.015120,-0.004640,0.052540,0.042300,0.041260,0.030000,0.026425,-0.024265,0.025233


# Results Panel A
The results below are already expressed as excess returns because when computing them were used excess returns not raw returns. Were computed the annualised excess return, annualised volatility, sharpe ratio, and the alpha and beta with respect to the CAPM.

In [378]:
stats(rets_vol_strat, 12, ff_factors.loc["1929-07":"2023-01","Mkt_RF"])

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D1_minus_ten,Univ
Annualized Return,0.087461,0.091499,0.093676,0.094447,0.080061,0.093002,0.089023,0.102932,0.090411,0.052731,-0.056253,0.092389
Annualized Vol,0.178619,0.217869,0.233169,0.252790,0.255414,0.281743,0.268239,0.291556,0.314300,0.373378,0.262767,0.250012
Sharpe Ratio,0.489652,0.419973,0.401753,0.373619,0.313455,0.330095,0.331881,0.353044,0.287659,0.141226,-0.214080,0.369540
alpha,0.034800,0.033600,0.033600,0.032400,0.018000,0.027600,0.026400,0.038400,0.025200,0.001200,0.034800,0.027600
t_value,4.169100,3.025100,2.884000,2.483900,1.401500,1.903500,1.891600,2.409500,1.447900,0.030300,1.414000,2.419300
beta,0.853400,1.013700,1.101900,1.180400,1.196100,1.306600,1.247100,1.327300,1.425000,1.501600,-0.648200,1.203300
t_value,65.895900,58.600500,62.614900,59.464800,60.173900,57.813300,58.410400,53.919300,53.138900,38.015300,-17.360600,68.366600


Can be seen that most portfolios have similar performance with respect to their annualised return, but the bottom decile portfolio (this one significantly underperforms the others) so there seems to be a weak relation between historical volatility and the return on the following month if the last decile is not considered. The difference between the top and bottom decile is of around 3.5%.

However, if we switch the focus of the analysis from merely excess annualised returns to the return per unit of risk (sharpe ratio), interesting things arise. Can be seen a clear relationship in which the portfolios in the top deciles have higher sharpe ratio and as we go to the bottom deciles the sharpe ratio decreases substantially. The top decile portfolio has a sharpe ratio that is more than 3 times that of the bottom decile portfolio. The standard deviation is the main factor impacting the variation in the sharpe ratio.

Furthermore, can be seen the the alpha is significant and positive for top decile portfolios and it starts to decrease in significance as we move to the bottom decile portfolios, actually after the 5th decile the alpha is no longer significant (D8 being an exception) implying that the alpha is equal to zero, and therefore suggesting that those portfolios do not generate abnormal returns with respect to the CAPM. The alpha spread between the top versus the bottom decile is of around 3.4% annually.

With respect to the beta, can be seen that it tends to increase as we move from top decile portfolios to bottom decile portfolios. This makes sense because the beta is a measure of systematic risk, therefore the fact that top decile portfolios have lower volatility implies that their beta should be lower. On the other hand, this counters the intuition that higher beta stocks should produce higher returns, in this case this cannot be seen in the data. Actually, the low beta stocks produce higher risk adjusted returns than the high beta stocks.

In conclusion low-volatility stocks show higher risk adjusted returns than the universe portfolio and than higher-volatility stocks. Additionally, the volatility effect is another factor that is not explained by the single factor model: CAPM.

# Results Panel B

In [358]:
pos_months = rets_vol_strat.loc[rets_vol_strat["Univ"] > 0] # filtering on months with positive returns for the universe portfolio
neg_months = rets_vol_strat.loc[rets_vol_strat["Univ"] < 0] # filtering on months with negative returns for the universe portfolio

In [375]:
pos_avg_rets = pos_months.subtract(pos_months["Univ"],axis=0).mean() # calculating the monthly average over/under performance during positive months
neg_avg_rets = neg_months.subtract(neg_months["Univ"],axis=0).mean() # calculating the monthly average over/under performance during negative months
dd = rets_vol_strat.aggregate(lambda rets_vol_strat: drawdown(rets_vol_strat).Drawdown.min()) # calculating the maximum drawdown
panelb = pd.concat([pos_avg_rets, neg_avg_rets, dd], axis=1)
panelb.columns = ["Return_up", "Return_down", "Max Drawdown"]
panelb.T

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D1_minus_ten,Univ
Return_up,-0.015189,-0.007323,-0.003387,-0.000605,-0.000641,0.002866,0.002491,0.006762,0.008809,0.010803,-0.075504,0.000000
Return_down,0.018172,0.009006,0.004441,0.001420,-0.001137,-0.002509,-0.003231,-0.005679,-0.009867,-0.016583,0.082419,0.000000
Max Drawdown,-0.806141,-0.789763,-0.851767,-0.864186,-0.877313,-0.870302,-0.851239,-0.875701,-0.894849,-0.871187,-0.998966,-0.852023


In the above dataframe, can be seen the excess return with respect to the universe during months with overall positive returns and months with negative returns. Can be seen that when there are positive return, the top decile portfolios tend to underperform the universe portfolio, this makes sense because the beta of these portfolios is relatively lower.

On the other hand, the excess return of bottom decile portfolios with respect to the universe during months with negative returns is negative, whereas the top decile portfolios overperform during these months, again this makes due to the betas of each portfolio.

Finally, we can see as well the maximum drawdown, which refers to the maximum percentage loss from a peak to a bottom. This could be seen as a risk metric and can be seen that the top two decile portfolios are the ones exhibiting the lowest risk in terms of the historical drawdown.